In [3]:
import pandas as pd
import re
import openai
import os
import time
from dotenv import load_dotenv
import language_tool_python



In [4]:
#dotenv_path = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..', '.env'))
dotenv_path = "Users/pc/Documents/CSPB/CSPB 3112/nanoGPT/.env"
load_dotenv(dotenv_path)

False

In [5]:
#we need to clean our input data
def clean_txt(text):
    text = str(text).strip() #remove leading and trailing whitespaces
    text = re.sub(r'\s+', ' ', text) #fix any unneccessary whitespace
    text = re.sub(r'^\x00-\x7F]+', '', text) #remove any characters non-ASCII
    return text
#used to split our description on each row into the new column titled items
def split(text):
    txt_lst = text.split("-")
    return txt_lst[0]

def split2(txt):
    txt_lst = txt.split("-")
    return txt_lst[1]
    
#format the data in a conversational format
def format_row(row):
    return f"Customer: Tell me about the {row['item']}.\nAgent: The {row['item']} is {row['description']}.\n"

#some of our descriptions lack correct sentence structure which is an issue for training
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
def fix_grammar(txt):
    try:
        time.sleep(1)
        response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                            messages=[{"role": "system", "content": "Improve grammar and sentence structure of the following text."},
                                                      {"role": "user", "content": txt}], 
                                            temperature=0.2,
                                            max_tokens=100)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return txt

# Alternative method as compared to making API calls to chatGPT    
tool = language_tool_python.LanguageTool('en-US')
def corr_grammar(txt):
    matches = tool.check(txt)
    return language_tool_python.correct(txt, matches)


CalledProcessError: Command '['/usr/bin/java', '-version']' returned non-zero exit status 1.

In [17]:
#read in our data
df = pd.read_csv("sample-data.csv")

#our item is hidden in our description so we need to extract that and create a new column
df["item"] = df["description"].apply(split)
df["description"] = df["description"].apply(split2)
df.dropna(subset=["description", "item"], inplace=True)
#clean up our data
for col in ["id", "description", "item"]:
    df[col] = df[col].apply(clean_txt)
    

#finally reformat it so it can be written into a text file
formatted = df.apply(format_row, axis=1) #apply to row via axis=1

In [18]:
with open("train.txt", "w", encoding = 'utf-8') as f:
    f.writelines(formatted)
print("Completed step 1...")
with open("train.txt", "r") as f:
    lines = f.readlines()

fixed_lines = [fix_grammar(line) for line in lines if line.strip()]
print("Completed step 2...")
with open("new_train.txt", "w", encoding= 'utf-8') as f:
    for line in fixed_lines:
        f.write(line + "\n")
print("Your text file of training data is ready!")
        
    
        

Completed step 1...
Completed step 2...
Your text file of training data is ready!


In [2]:
#method 2 comparison
for col in ["id", "description", "item"]:
    df[col] = df[col].apply(corr_grammar)
formatted2 = df.apply(format_row, axis=1)

with open("method_2_train.txt", "w", encoding='utf-8') as file:
    file.writelines(formatted2)

NameError: name 'df' is not defined